# 特徴量設計検討ノート

このノートブックでは、競馬予想に使用する特徴量の検討・設計を行います。

## 目次
1. 現在取得可能なデータの確認
2. 競馬予想で有効な特徴量の調査
3. 特徴量候補のリストアップ
4. サンプルデータでの特徴量抽出テスト

---
## 1. 環境設定とデータ確認

In [ ]:
import os
import sys
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートへのパス設定
PROJECT_ROOT = Path('../../')
DATA_DIR = PROJECT_ROOT / 'data' / 'raceHTML'

print(f"データディレクトリ: {DATA_DIR.resolve()}")
print(f"年別フォルダ数: {len(list(DATA_DIR.glob('*')))}")

In [ ]:
# サンプルHTMLを読み込み
sample_html_path = DATA_DIR / '2024' / '202401010101.html'
with open(sample_html_path, 'r', encoding='utf-8') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')
print(f"HTMLファイル読み込み完了: {sample_html_path.name}")

---
## 2. HTMLから取得可能なデータ項目の確認

In [ ]:
# レース情報の取得
race_data = soup.find('dl', class_='racedata')
print("=== レース基本情報 ===")
if race_data:
    race_name = race_data.find('h1').text if race_data.find('h1') else 'N/A'
    race_info = race_data.find('span').text if race_data.find('span') else 'N/A'
    print(f"レース名: {race_name}")
    print(f"レース情報: {race_info}")

In [ ]:
# 結果テーブルの取得
result_table = soup.find('table', class_='race_table_01')
if result_table:
    # ヘッダー取得
    headers = [th.text.strip().replace('\n', '') for th in result_table.find_all('th')]
    print("=== テーブルヘッダー ===")
    for i, h in enumerate(headers):
        print(f"{i+1}. {h}")

In [ ]:
# 各馬のデータを抽出
rows = result_table.find_all('tr')[1:]  # ヘッダー行をスキップ
print(f"\n出走頭数: {len(rows)}頭")
print("\n=== 1着馬のデータサンプル ===")

if rows:
    first_row = rows[0]
    cells = first_row.find_all('td')
    
    # 各セルの内容を確認
    for i, cell in enumerate(cells):
        text = cell.text.strip().replace('\n', ' ')
        link = cell.find('a')
        if link:
            href = link.get('href', '')
            print(f"セル{i}: {text} (リンク: {href[:50]}...)")
        else:
            print(f"セル{i}: {text}")

---
## 3. 特徴量候補のリストアップ

### 3.1 HTMLから直接取得可能な特徴量

| カテゴリ | 特徴量 | 説明 |
|---------|--------|------|
| **レース情報** | race_id | レースID（12桁） |
| | venue | 競馬場（01-10） |
| | distance | 距離（m） |
| | surface | 芝/ダート |
| | direction | 右回り/左回り |
| | weather | 天候 |
| | track_condition | 馬場状態（良/稍重/重/不良） |
| | race_class | クラス |
| **馬情報** | horse_id | 馬ID |
| | horse_name | 馬名 |
| | sex | 性別 |
| | age | 年齢 |
| | weight | 斤量 |
| | horse_weight | 馬体重 |
| | weight_change | 馬体重増減 |
| **騎手/調教師** | jockey_id | 騎手ID |
| | trainer_id | 調教師ID |
| **レース結果** | finish_position | 着順 |
| | time | タイム |
| | margin | 着差 |
| | last_3f | 上がり3F |
| | passing | 通過順 |
| | odds | 単勝オッズ |
| | popularity | 人気順 |

### 3.2 計算が必要な特徴量（過去成績ベース）

| カテゴリ | 特徴量 | 説明 | 計算方法 |
|---------|--------|------|----------|
| **馬の過去成績** | win_rate | 勝率 | 1着回数 / 出走回数 |
| | place_rate | 複勝率 | 3着以内回数 / 出走回数 |
| | avg_finish | 平均着順 | 着順の平均 |
| | last_finish | 前走着順 | 直近レースの着順 |
| | days_since_last | 前走からの間隔 | 日数 |
| | avg_last_3f | 平均上がり3F | |
| | best_last_3f | 最速上がり3F | |
| | venue_win_rate | 競馬場別勝率 | |
| | distance_win_rate | 距離別勝率 | |
| | surface_win_rate | 芝/ダート別勝率 | |
| **騎手成績** | jockey_win_rate | 騎手勝率 | |
| | jockey_place_rate | 騎手複勝率 | |
| | jockey_venue_rate | 騎手競馬場別成績 | |
| **調教師成績** | trainer_win_rate | 調教師勝率 | |
| | trainer_place_rate | 調教師複勝率 | |
| **コンビ成績** | horse_jockey_rate | 馬×騎手組み合わせ成績 | |

### 3.3 レース当日の相対的特徴量

| カテゴリ | 特徴量 | 説明 |
|---------|--------|------|
| **人気関連** | popularity_rank | 人気順位 |
| | odds_ratio | オッズ比（最低オッズとの比） |
| **馬体重関連** | weight_vs_avg | 馬体重（出走馬平均との差） |
| | weight_class | 馬体重クラス（軽量/中量/重量） |
| **枠順関連** | gate_position | 枠番 |
| | gate_inside | 内枠フラグ（1-3枠） |
| | gate_outside | 外枠フラグ（6-8枠） |

---
## 4. HTMLパース関数のプロトタイプ

In [ ]:
def parse_race_html(html_path):
    """
    レースHTMLをパースして構造化データを抽出する
    
    Returns:
        dict: race_info, horses の辞書
    """
    with open(html_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
    
    # race_idをファイル名から取得
    race_id = html_path.stem
    
    # レース情報
    race_info = {'race_id': race_id}
    
    # レース詳細
    race_data = soup.find('dl', class_='racedata')
    if race_data:
        race_info['race_name'] = race_data.find('h1').text.strip() if race_data.find('h1') else ''
        
        span = race_data.find('span')
        if span:
            info_text = span.text.strip()
            # 距離・コース抽出
            match = re.search(r'(芝|ダート)(右|左)?(\d+)m', info_text)
            if match:
                race_info['surface'] = match.group(1)
                race_info['direction'] = match.group(2) or ''
                race_info['distance'] = int(match.group(3))
            
            # 天候
            weather_match = re.search(r'天候 : (\S+)', info_text)
            if weather_match:
                race_info['weather'] = weather_match.group(1)
            
            # 馬場状態
            condition_match = re.search(r'(芝|ダート) : (\S+)', info_text)
            if condition_match:
                race_info['track_condition'] = condition_match.group(2)
    
    # 日付情報
    date_elem = soup.find('p', class_='smalltxt')
    if date_elem:
        date_match = re.search(r'(\d{4})年(\d{2})月(\d{2})日', date_elem.text)
        if date_match:
            race_info['date'] = f"{date_match.group(1)}-{date_match.group(2)}-{date_match.group(3)}"
    
    # 出走馬情報
    horses = []
    result_table = soup.find('table', class_='race_table_01')
    if result_table:
        for row in result_table.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) < 10:
                continue
            
            horse = {
                'race_id': race_id,
                'finish_position': cells[0].text.strip(),
                'gate_number': cells[1].text.strip(),
                'horse_number': cells[2].text.strip(),
            }
            
            # 馬名とID
            horse_link = cells[3].find('a')
            if horse_link:
                horse['horse_name'] = horse_link.text.strip()
                href = horse_link.get('href', '')
                horse_id_match = re.search(r'/horse/(\d+)/', href)
                if horse_id_match:
                    horse['horse_id'] = horse_id_match.group(1)
            
            # 性齢
            sex_age = cells[4].text.strip()
            if sex_age:
                horse['sex'] = sex_age[0] if sex_age else ''
                horse['age'] = int(sex_age[1:]) if len(sex_age) > 1 else 0
            
            # 斤量
            horse['weight'] = float(cells[5].text.strip()) if cells[5].text.strip() else 0
            
            # 騎手
            jockey_link = cells[6].find('a')
            if jockey_link:
                horse['jockey_name'] = jockey_link.text.strip()
                href = jockey_link.get('href', '')
                jockey_id_match = re.search(r'/jockey/result/recent/(\d+)/', href)
                if jockey_id_match:
                    horse['jockey_id'] = jockey_id_match.group(1)
            
            # タイム
            horse['time'] = cells[7].text.strip()
            
            horses.append(horse)
    
    return {'race_info': race_info, 'horses': horses}

# テスト実行
result = parse_race_html(sample_html_path)
print("=== レース情報 ===")
for k, v in result['race_info'].items():
    print(f"  {k}: {v}")

print(f"\n=== 出走馬数: {len(result['horses'])}頭 ===")
if result['horses']:
    print("1着馬:", result['horses'][0])

---
## 5. 次のステップ

1. **HTMLパーサーの完成** (`src/scraper/parser.py`)
   - 全データ項目の抽出
   - エラーハンドリング

2. **特徴量計算モジュール** (`src/features/`)
   - 過去成績の集計
   - 騎手・調教師成績

3. **データパイプライン構築** (`S02_feature_engineering.ipynb`)
   - 全HTMLの処理
   - 特徴量データセット生成